In [1]:
import os,sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import warnings
warnings.filterwarnings('ignore')

import re, nltk, string

import sqlite3 #input doc is sqlite file

C:\Users\achyu\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [78]:
from nltk.stem.wordnet import WordNetLemmatizer
import sklearn
from sklearn.feature_extraction.text import CountVectorizer #BOW
from sklearn.feature_extraction.text import TfidfVectorizer #Tfidf
#It performs both the tokenization (converting text into words or n-grams) and the TF-IDF transformation in a single step.
from sklearn.feature_extraction.text import TfidfTransformer
#TfidfTransformer is used when you already have a count matrix (e.g., obtained using CountVectorizer) and want to transform 
  #it into a TF-IDF representation.

from sklearn.metrics import confusion_matrix,classification_report,accuracy_score,roc_curve,auc

#deep learning packages

from gensim.models import Word2Vec
#Word2Vec is a class in Gensim that is used for training Word Embeddings using the Word2Vec algorithm.
from gensim.models import KeyedVectors
#KeyedVectors is a class in Gensim that provides an interface to access and manipulate word vectors.p`p

In [3]:
# importing dataset by using sqlite3 as the input doc is in sqlite format

con = sqlite3.connect("D:\\Learnbay\\NLP\\Notes\\database.sqlite")
con

In [4]:
filtered_data = pd.read_sql_query("""select * from reviews where score """,con)
filtered_data.shape

(568454, 10)

In [5]:
filtered_data['Score'].value_counts()

5    363122
4     80655
1     52268
3     42640
2     29769
Name: Score, dtype: int64

In [6]:
filtered_data1 = pd.read_sql_query("""select * from reviews where score !=3 """,con)
filtered_data1.shape

(525814, 10)

In [7]:
filtered_data1['Score'].value_counts()

5    363122
4     80655
1     52268
2     29769
Name: Score, dtype: int64

In [8]:
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [9]:
filtered_data3 = pd.read_sql_query("""select * from reviews LIMIT 5000 """,con)
filtered_data3.shape

(5000, 10)

In [10]:
def partition(x):
    if x<3:
        return 3
    elif x==3:
        return 2
    else:
        return 1

actualScore = filtered_data3['Score']
PositiveNegativeNeutral = actualScore.map(partition)
filtered_data3['Score'] = PositiveNegativeNeutral
print("No of data points in our data :",filtered_data3.shape)

No of data points in our data : (5000, 10)


In [11]:
filtered_data3.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,3,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,3,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,1,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [12]:
filtered_data3['Score'].value_counts()

1    3846
3     759
2     395
Name: Score, dtype: int64

In [13]:
final = filtered_data3[filtered_data3.HelpfulnessNumerator<=filtered_data3.HelpfulnessDenominator]
final.shape

(5000, 10)

In [14]:
final.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,3,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,3,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,1,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [15]:
final['Text'][100]
#there is a html format in text

'The mouth says, "How do I love thee, let me count the ways..."<br />If you like apple products a must have item.  The only draw back, shipping cost.  These are very heavy.'

In [16]:
final['Text'][10]

"I don't know if it's the cactus or the tequila or just the unique combination of ingredients, but the flavour of this hot sauce makes it one of a kind!  We picked up a bottle once on a trip we were on and brought it back home with us and were totally blown away!  When we realized that we simply couldn't find it anywhere in our city we were bummed.<br /><br />Now, because of the magic of the internet, we have a case of the sauce and are ecstatic because of it.<br /><br />If you love hot sauce..I mean really love hot sauce, but don't want a sauce that tastelessly burns your throat, grab a bottle of Tequila Picante Gourmet de Inclan.  Just realize that once you taste it, you will never want to use any other sauce.<br /><br />Thank you for the personal, incredible service!"

In [17]:
final['Text'][5]
#symbols and numbers will be there too in the text

'I got a wild hair for taffy and ordered this five pound bag. The taffy was all very enjoyable with many flavors: watermelon, root beer, melon, peppermint, grape, etc. My only complaint is there was a bit too much red/black licorice-flavored pieces (just not my particular favorites). Between me, my kids, and my husband, this lasted only two weeks! I would recommend this brand of taffy -- it was a delightful treat.'

In [18]:
sent_0 = final['Text'][50]
sent_1 = final['Text'][500]
sent_2 = final['Text'][1550]
sent_3 = final['Text'][3500]
sent_4 = final['Text'][4500]
print(sent_0)
print("**********************3")
print(sent_1)
print("**********************3")
print(sent_2)
print("**********************3")
print(sent_3)
print("**********************3")
print(sent_4)
print("**********************3")

This oatmeal is not good. Its mushy, soft, I don't like it. Quaker Oats is the way to go.
**********************3
I bought these for the office and no one can put them down. I'm ordering more today and it has only been 2 weeks. Didn't have to shop for them either, Amazon brought it to my door!
**********************3
I liked this so much that I re-ordered and will keep<br /><br />re-ordering.
**********************3
This is a very mild hot cocoa.  Perfect for those winter evenings or even good in the morning instead of coffee.  We love it anytime.  Next time I will try the dark chocolate.
**********************3
I have a pretty wide appreciation for acceptable coffee, but in my opinion, this stuff is mediocre coffee. I drink both regular and decaf on a daily basis and to me, this tasted like institutional cafeteria coffee. I am normally a fan of the "Swiss water decaffeination process" decaf coffees, and was disappointed by the quality of this "gourmet" coffee.<br /><br />With that sai

In [19]:
#remove -symbols,numbers,http format,stopwords,converting uppercase to lowercase

In [20]:
word_stop = nltk.corpus.stopwords.words('english')
word_stop

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [21]:
stopwords = set(['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each',
 'few',
 'more',
 'most',
 'other',
 'some',
 'such',
 'no',
 'nor',
 'not',
 'only',
 'own',
 'same',
 'so',
 'than',
 'too',
 'very',
 's',
 't',
 'can',
 'will',
 'just',
 'don',
 "don't",
 'should',
 "should've",
 'now',
 'd',
 'll',
 'm',
 'o',
 're',
 've',
 'y',
 'ain',
 'aren',
 "aren't",
 'couldn',
 "couldn't",
 'didn',
 "didn't",
 'doesn',
 "doesn't",
 'hadn',
 "hadn't",
 'hasn',
 "hasn't",
 'haven',
 "haven't",
 'isn',
 "isn't",
 'ma',
 'mightn',
 "mightn't",
 'mustn',
 "mustn't",
 'needn',
 "needn't",
 'shan',
 "shan't",
 'shouldn',
 "shouldn't",
 'wasn',
 "wasn't",
 'weren',
 "weren't",
 'won',
 "won't",
 'wouldn',
 "wouldn't","br"])

In [22]:
import contractions # converts short form to easy understandable format
# it's => it is 
#don't =>do not
#i'll => i will

In [38]:
from tqdm import tqdm  # display progress bars for iterations
from bs4 import BeautifulSoup #Imports the BeautifulSoup library for webscrapping &HTML parsing
from nltk.stem import WordNetLemmatizer
wordnet = WordNetLemmatizer()
#stopwords = set(stopwords.words('english'))

preprocessed_reviews = []

for sentence in tqdm(final['Text'].values):
    sentence = re.sub(r"http\S+","",sentence)
    sentence = contractions.fix(sentence)
    sentence = BeautifulSoup(sentence,'lxml',).get_text()
    sentence = re.sub("\S*\d\S*","",sentence).strip()
    sentence = re.sub("[^a-zA-Z]+"," ",sentence)
    #sentence = [wordnet.lemmatize(word) for word in sentence.split() if word.lower() not in stopwords]
    sentence = ' '.join(e.lower() for e in sentence.split() if e.lower() not in stopwords)
    preprocessed_reviews.append(sentence.strip())


100%|████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:02<00:00, 1900.57it/s]


In [39]:

"""
# Examples of above codes

1.
data = {'Text': ['This is sentence 1', 'Another sentence', 'And one more']}
df = pd.DataFrame(data)

for sentence in df['Text']:
    print(type(sentence))  # Output: <class 'str'>

# With .values
for sentence in df['Text'].values:
    print(type(sentence))  # Output: <class 'numpy.str_'>

2.sentence = re.sub(r"http\S+","",sentence)
sentence = "Check out this website: http://www.example.com for more information."
Check out this website:  for more information.

3sentence = BeautifulSoup(sentence, 'lxml').get_text()
sentence = "<p>This is <b>bold</b> and <i>italic</i>.</p>"
This is bold and italic.

4.sentence = re.sub("\S*\d\S*","",sentence).strip()
sentence = re.sub("\S*\d\S*","",sentence).strip()
Remove  digits from this example sentence.


"""

'\n# Examples of above codes\n\n1.\ndata = {\'Text\': [\'This is sentence 1\', \'Another sentence\', \'And one more\']}\ndf = pd.DataFrame(data)\n\nfor sentence in df[\'Text\']:\n    print(type(sentence))  # Output: <class \'str\'>\n\n# With .values\nfor sentence in df[\'Text\'].values:\n    print(type(sentence))  # Output: <class \'numpy.str_\'>\n\n2.sentence = re.sub(r"http\\S+","",sentence)\nsentence = "Check out this website: http://www.example.com for more information."\nCheck out this website:  for more information.\n\n3sentence = BeautifulSoup(sentence, \'lxml\').get_text()\nsentence = "<p>This is <b>bold</b> and <i>italic</i>.</p>"\nThis is bold and italic.\n\n4.sentence = re.sub("\\S*\\d\\S*","",sentence).strip()\nsentence = re.sub("\\S*\\d\\S*","",sentence).strip()\nRemove  digits from this example sentence.\n\n\n'

In [40]:
sent_0 = preprocessed_reviews[50]
sent_1 = preprocessed_reviews[500]
sent_2 = preprocessed_reviews[1550]
sent_3 = preprocessed_reviews[3500]
sent_4 = preprocessed_reviews[4500]
print(sent_0)
print("**********************"*3)
print(sent_1)
print("**********************"*3)
print(sent_2)
print("**********************"*3)
print(sent_3)
print("**********************"*3)
print(sent_4)
print("**********************"*3)

oatmeal good mushy soft like quaker oats way go
******************************************************************
bought office one put ordering today weeks shop either amazon brought door
******************************************************************
liked much ordered keepre ordering
******************************************************************
mild hot cocoa perfect winter evenings even good morning instead coffee love anytime next time try dark chocolate
******************************************************************
pretty wide appreciation acceptable coffee opinion stuff mediocre coffee drink regular decaf daily basis tasted like institutional cafeteria coffee normally fan swiss water decaffeination process decaf coffees disappointed quality gourmet coffee said try twice give fair review first time prepared way normally make coffee strong side second time followed directions exactly times using filtered water quality coffee maker uses melitta style cone filter taste 

# TF-IDF

In [57]:
tf_idf_vect = TfidfVectorizer(ngram_range=(1, 2),max_df=0.85, min_df=2,)
tf_idf_vect.fit(preprocessed_reviews)
print("Some Features Name :",tf_idf_vect.get_feature_names()[0:20])

print("*****************"*3)

final_counts_tfidf = tf_idf_vect.transform(preprocessed_reviews)

print("The type of count Vectorizer :",type(final_counts_tfidf))

print("The shape of the Text TFIDF vectorizer :", final_counts_tfidf.get_shape())

print("The number of unique words :", final_counts_tfidf.get_shape()[1])

Some Features Name : ['ability', 'able', 'able buy', 'able drink', 'able eat', 'able feed', 'able find', 'able get', 'able give', 'able go', 'able make', 'able order', 'able purchase', 'able save', 'able share', 'able try', 'able use', 'absence', 'absent', 'absolute']
***************************************************
The type of count Vectorizer : <class 'scipy.sparse._csr.csr_matrix'>
The shape of the Text TFIDF vectorizer : (5000, 25632)
The number of unique words : 25632


In [58]:
final_counts_tfidf = tf_idf_vect.transform(preprocessed_reviews).toarray()

In [59]:
pd.DataFrame(final_counts_tfidf).head()

,0,1,2,3,4,5,6,7,8,9,...,25622,25623,25624,25625,25626,25627,25628,25629,25630,25631
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
y = final['Score']

In [61]:
y.head()

0    1
1    3
2    1
3    3
4    1
Name: Score, dtype: int64

In [62]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(final_counts_tfidf,y,test_size =0.2,
                                                random_state =5,stratify = y)

In [63]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(x_train,y_train)

RandomForestClassifier()

In [64]:
y_pred_train = rf.predict(x_train)
y_pred_test = rf.predict(x_test)

In [65]:
print("Train Accuracy:",accuracy_score(y_train,y_pred_train))
print("*"*15)
print("Test Accuracy :", accuracy_score(y_test,y_pred_test))

Train Accuracy: 1.0
***************
Test Accuracy : 0.788


In [66]:
print("Train Accuracy:",classification_report(y_train,y_pred_train))
print("*"*15)
print("Test Accuracy :", classification_report(y_test,y_pred_test))

Train Accuracy:               precision    recall  f1-score   support

           1       1.00      1.00      1.00      3077
           2       1.00      1.00      1.00       316
           3       1.00      1.00      1.00       607

    accuracy                           1.00      4000
   macro avg       1.00      1.00      1.00      4000
weighted avg       1.00      1.00      1.00      4000

***************
Test Accuracy :               precision    recall  f1-score   support

           1       0.79      1.00      0.88       769
           2       1.00      0.01      0.02        79
           3       0.86      0.12      0.21       152

    accuracy                           0.79      1000
   macro avg       0.88      0.38      0.37      1000
weighted avg       0.81      0.79      0.71      1000



# PCA

In [52]:
from sklearn.decomposition import PCA


In [53]:
pca = PCA(n_components=None)
x_train_n = pca.fit_transform(x_train)
x_test_n = pca.fit_transform(x_test)

In [68]:
explained_variance = pca.explained_variance_ratio_

In [69]:
explained_variance

array([1.00327574e-02, 8.38291865e-03, 7.49444836e-03, 7.00915460e-03,
       6.89386448e-03, 5.75856339e-03, 5.20107135e-03, 4.84852825e-03,
       4.33113670e-03, 4.22619304e-03, 4.13271908e-03, 3.94428551e-03,
       3.91472697e-03, 3.82925107e-03, 3.78512740e-03, 3.68062785e-03,
       3.65164615e-03, 3.51826243e-03, 3.43665556e-03, 3.40751523e-03,
       3.37169184e-03, 3.32817589e-03, 3.29652550e-03, 3.20825226e-03,
       3.16670957e-03, 3.13552330e-03, 3.10258562e-03, 3.08502185e-03,
       3.05402662e-03, 3.01908572e-03, 2.95830223e-03, 2.94283775e-03,
       2.91272376e-03, 2.90145013e-03, 2.86139328e-03, 2.84865946e-03,
       2.83009223e-03, 2.79273199e-03, 2.74993872e-03, 2.73848650e-03,
       2.72029642e-03, 2.69995767e-03, 2.69282243e-03, 2.67862260e-03,
       2.65238535e-03, 2.62984501e-03, 2.60573528e-03, 2.59686083e-03,
       2.58333704e-03, 2.54578386e-03, 2.54192864e-03, 2.52578266e-03,
       2.51289130e-03, 2.47699052e-03, 2.47015347e-03, 2.45522379e-03,
      

In [70]:
len(explained_variance)

1000

In [71]:
pca = PCA(n_components=100)
x_train_100 = pca.fit_transform(x_train)
x_test_100 = pca.fit_transform(x_test)

In [73]:
rf = RandomForestClassifier()
rf.fit(x_train_100, y_train)

RandomForestClassifier()

In [74]:
y_pred_train = rf.predict(x_train_100)
y_pred_test = rf.predict(x_test_100)

In [75]:
print("Train Accuracy:",accuracy_score(y_train,y_pred_train))
print("*"*15)
print("Test Accuracy :", accuracy_score(y_test,y_pred_test))

Train Accuracy: 1.0
***************
Test Accuracy : 0.77


In [76]:
print("Train Accuracy:",classification_report(y_train,y_pred_train))
print("*"*15)
print("Test Accuracy :", classification_report(y_test,y_pred_test))

Train Accuracy:               precision    recall  f1-score   support

           1       1.00      1.00      1.00      3077
           2       1.00      1.00      1.00       316
           3       1.00      1.00      1.00       607

    accuracy                           1.00      4000
   macro avg       1.00      1.00      1.00      4000
weighted avg       1.00      1.00      1.00      4000

***************
Test Accuracy :               precision    recall  f1-score   support

           1       0.77      1.00      0.87       769
           2       0.00      0.00      0.00        79
           3       0.33      0.01      0.01       152

    accuracy                           0.77      1000
   macro avg       0.37      0.34      0.29      1000
weighted avg       0.64      0.77      0.67      1000



# Word2Vec-Self Trained Model

In [83]:
import gensim
from gensim.models import Word2Vec

preprocessed_reviews1 = pd.DataFrame(preprocessed_reviews)

model =gensim.models.Word2Vec(window = 10, min_count = 2, sg=0)
model.build_vocab(preprocessed_reviews1,total_examples = model.corpus_count,epochs = model.epochs)

TypeError: 'int' object is not iterable

# Word2Vec-Pre Trained Model

In [ ]:
#this is has to run only in GPU device
i = 0
list_of_sentence = []
for sentence in preprocessed_reviews:
    list_of_sentence.append(sentence.split())
    
is_your_ram_gt_16g = False
want_to_use_google_w2v = False
want_to_train_w2v =True

if want_to_train_w2v:
    w2v_model =Word2Vec(list_of_sentence,window =10, min_count =2,sg=0)
    print(w2v_model.wv.most_similar('good'))
    print("*"*20)
    print(w2vmodel.wv.most_similar('bad'))
elif want_to_use_google_w2v and is_your_ram_gt_16g:
    if os.path.isfile("GoogleNews-vectors-negative300.bin")